In [ ]:
# Demo
from sagemaker.estimator import Estimator
import sagemaker
from sagemaker import image_uris


role = sagemaker.get_execution_role()
bucket = 'gallstone-prediction'
xgboost_image = image_uris.retrieve(
    framework='xgboost',
    region='ap-south-1',
    version='1.5-1'
)

estimator = Estimator(
    image_uri=xgboost_image,
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    entry_point='train.py',
    source_dir='.',
    base_job_name='gallstone-xgb'
)

estimator.fit({'train': f's3://{bucket}/train.csv',
               'val': f's3://{bucket}/val.csv'})


In [ ]:
# Import the model
from sagemaker.model import Model

model_data = 's3://sagemaker-ap-south-1-XXXXXXXXXXXX/gallstone-xgb-2025-08-31-00-11-14-026/output/model.tar.gz'

xgb_model = Model(image_uri=xgboost_image,
                  model_data=model_data,
                  role=role,
                  entry_point='inference.py',
                  source_dir='./',
                  sagemaker_session=sagemaker.Session()
                 )


In [ ]:
#Deploy the endpoint

predictor = xgb_model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name='gallstone'
)

In [ ]:
predictor.delete_endpoint(EndpointName='gallstone')